In [ ]:
!pip install --upgrade pip
!pip install numpy
!pip install pandas
!pip install xlrd

In [1]:
debug = 1
import numpy as np
import pandas
pandas.set_option('display.max_colwidth', -1)
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

data_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()

# 1-6, 139-150
want_sequences = []
for i in range(1,7):
    want_sequences.append(i)
for i in range(139,151):
    want_sequences.append(i)

#print(want_sequences)

#for i in range(1,167):
for i in want_sequences:
    if (debug >= 1):
#        if (np.mod(i, 10) == 0):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_data_df = pandas.DataFrame()
    this_seq_metadata_filename = '/home/idies/workspace/raddick_acs_data/metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_in_cols_df = this_seq_metadata_in_cols_df.dropna(axis=1)
    
    if (i == 1):
        this_seq_metadata_in_cols_df.columns = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','B00001_001_UW','B00001_002_UW']
    else:
        this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    if (i == 1):
        this_seq_metadata_df['variable'] = this_seq_metadata_in_cols_df.columns
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1] 
    else:
        this_seq_metadata_df['sequence_number'] = i
    
    for onestate in states:
        statefilename = '/home/idies/workspace/raddick_acs_data/data/seq{0:.0f}/e20161{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8')
        this_seq_data_df = this_seq_data_df.append(onestate_df)
        
    this_seq_data_df.columns = this_seq_metadata_in_cols_df.columns
    
    this_seq_data_df = this_seq_data_df.drop(['SEQUENCE'], axis=1)
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_data_df = this_seq_data_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

#    if (debug >= 2):
#        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    data_df = pandas.concat((data_df, this_seq_data_df), axis=1)

if (debug >= 2):
    print('Creating pseudo geoid...')
data_df['PSEUDO_GEOID'] = data_df['STUSAB'].str.upper() + data_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))

#metadata_df.loc['PSEUDO_GEOID'] = ['PSEUDO_GEOID', 'Temporary ID constructed from the STUSAB and the 7-digit zero-padded LOGRECNO to use to look up the geoid later.', 0]

if (debug >= 2):
    print('Indexing...')
data_df = data_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geoid_df = pandas.read_excel('metadata/geoid.xlsx', encoding='utf-8')
geoid_df = geoid_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')
data_df = data_df.join(geoid_df[['GEOID','NAME']])
#data_df = data_df.rename(columns={'NAME': 'GEO_NAME'})

#metadata_df['GEOID'] = 'Geography identifier'
#metadata_df['NAME'] = 'Name of geography unit'

if (debug >= 2):
    print('Resetting index...')
#metadata_df = metadata_df.set_index('GEOID')

metadata_df.loc['GEOID'] = ['GEOID', 'Geography identifier (also functions as index of data tables)', 0]
metadata_df.loc['NAME'] = ['NAME', 'Name of geography region', 0]


if (debug >= 2):
    print('Rearranging columns')
cols = data_df.columns.tolist()

data_df = data_df.set_index('GEOID', drop=False)
data_df.index.name = ''

cols = cols[-2:] + cols[:-2]
data_df = data_df[cols]

metadata_rows = cols
#metadata_rows = ['GEOID'] + cols
metadata_df = metadata_df.reindex(metadata_rows)

#metadata_df = metadata_df.set_index('variable', drop=False)

if (debug >= 1):
    print('Writing variables file...')
metadata_df.to_csv('/home/idies/workspace/raddick_acs_data/variables_clara.csv', encoding='utf-8')

if (debug >= 1):
    print('Writing data (estimates) file...')
data_df.to_csv('/home/idies/workspace/raddick_acs_data/data_acs2016_clara.csv', encoding='utf-8')

print('Done!')
#cols
#data_df.sample(1)
#metadata_df

Importing data for sequence 1...
Importing data for sequence 2...
Importing data for sequence 3...
Importing data for sequence 4...
Importing data for sequence 5...
Importing data for sequence 6...
Importing data for sequence 139...
Importing data for sequence 140...
Importing data for sequence 141...
Importing data for sequence 142...
Importing data for sequence 143...
Importing data for sequence 144...
Importing data for sequence 145...
Importing data for sequence 146...
Importing data for sequence 147...
Importing data for sequence 148...
Importing data for sequence 149...
Importing data for sequence 150...
Writing variables file...
Writing data (estimates) file...
Done!


In [ ]:
debug = 1

import numpy as np
import pandas
pandas.set_option('display.max_colwidth', -1)
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

# 1-6, 139-150
want_sequences = []
for i in range(1,7):
    want_sequences.append(i)
for i in range(139,151):
    want_sequences.append(i)

error_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()

#for i in range(1,167):
for i in want_sequences:
    if (debug >= 1):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_error_df = pandas.DataFrame()
    this_seq_metadata_filename = '/home/idies/workspace/raddick_acs_data/metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_in_cols_df = this_seq_metadata_in_cols_df.dropna(axis=1)
    
    if (i == 1):
        this_seq_metadata_in_cols_df.columns = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','B00001_001_UW','B00001_002_UW']
    else:
        this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    if (i == 1):
        this_seq_metadata_df['variable'] = this_seq_metadata_in_cols_df.columns
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1] 
    else:
        this_seq_metadata_df['sequence_number'] = i
    
    for onestate in states:
        statefilename = '/home/idies/workspace/raddick_acs_data/data/seq{0:.0f}/m20161{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8')
        this_seq_error_df = this_seq_error_df.append(onestate_df)
        
    this_seq_error_df.columns = this_seq_metadata_in_cols_df.columns
    
    this_seq_error_df = this_seq_error_df.drop(['SEQUENCE'], axis=1)
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_error_df = this_seq_error_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

#    if (debug >= 2):
#        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    error_df = pandas.concat((error_df, this_seq_error_df), axis=1)

if (debug >= 2):
    print('Creating pseudo geoid...')
error_df['PSEUDO_GEOID'] = error_df['STUSAB'].str.upper() + error_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))


if (debug >= 2):
    print('Indexing...')
error_df = error_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geoid_df = pandas.read_excel('metadata/geoid.xlsx', encoding='utf-8')
geoid_df = geoid_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')
error_df = error_df.join(geoid_df[['GEOID','NAME']])
#data_df = data_df.rename(columns={'NAME': 'GEO_NAME'})

#metadata_df['GEOID'] = 'Geography identifier'
#metadata_df['NAME'] = 'Name of geography unit'

if (debug >= 2):
    print('Resetting index...')
#metadata_df = metadata_df.set_index('GEOID')

metadata_df.loc['GEOID'] = ['GEOID', 'Geography identifier (also functions as index of data tables)', 0]
metadata_df.loc['NAME'] = ['NAME', 'Name of geography region', 0]


if (debug >= 2):
    print('Rearranging columns')
cols = error_df.columns.tolist()

error_df = error_df.set_index('GEOID', drop=False)

cols = cols[-2:] + cols[:-2]
error_df = error_df[cols]

metadata_rows = cols
#metadata_rows = ['GEOID'] + cols
metadata_df = metadata_df.reindex(metadata_rows)

#metadata_df = metadata_df.set_index('variable', drop=False)

#if (debug >= 1):
#    print('Writing variables file...')
#metadata_df.to_csv('/home/idies/workspace/raddick_acs_data/variables_acs2016_1_2.csv', encoding='utf-8')
error_df.index.name = ''


if (debug >= 1):
    print('Writing margins of error file...')
error_df.to_csv('/home/idies/workspace/raddick_acs_data/error_acs2016_clara.csv', encoding='utf-8')

print('Done!')
#cols
#error_df.sample(1)
#metadata_df

In [2]:
debug = 1
import numpy as np
import pandas


states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

geo_df = pandas.DataFrame()
geo_metadata_df = pandas.DataFrame()

print('Importing geography data...')

geo_metadata_filename = 'metadata/2016_SFGeoFileTemplate.xls'
geo_metadata_df = pandas.read_excel(geo_metadata_filename, header=None, encoding='utf-8')
geo_metadata_df = geo_metadata_df.dropna(axis=1)
geo_metadata_df.columns = geo_metadata_df.loc[0]
geo_metadata_df = geo_metadata_df.drop(0, axis=0)
geo_metadata_df.index = ['description']

for onestate in states:
    statefilename = 'geography/g20161{0:}.csv'.format(onestate)
    if (debug >= 2):
        print(statefilename)
    onestate_df = pandas.read_csv(statefilename, header=None, encoding='utf-8')
    geo_df = geo_df.append(onestate_df)

geo_df.columns = geo_metadata_df.columns
geo_df = geo_df.set_index('GEOID', drop=False)
geo_df.index.name = ''

geo_metadata_df = geo_metadata_df.T
geo_metadata_df = geo_metadata_df.reset_index()
geo_metadata_df.columns = ['variable', 'description']
geo_metadata_df = geo_metadata_df.set_index('variable', drop=False)
geo_metadata_df.index.name = ''

#geo_df = geo_df[columns=new_column_list]

#new_geo_df.tail(1)
#geo_metadata_df

#newcols
#df = df[cols]
print('Writing geography variables file...')
geo_metadata_df.to_csv('/home/idies/workspace/raddick_acs_data/geo_variables_acs2016_clara.csv', encoding='utf-8')

print('Writing geography file...')
geo_df.to_csv('/home/idies/workspace/raddick_acs_data/geo_acs2016_clara.csv', encoding='utf-8')

print('Done!')

Importing geography data...
Writing geography variables file...
Writing geography file...
Done!


In [ ]:

# reorder columns
new_column_list = []
new_column_list.append(geo_metadata_df.columns[len(geo_metadata_df.columns)-5])
for i in range(0,len(geo_metadata_df.columns)-5):
    new_column_list.append(geo_metadata_df.columns[i])
for j in range(len(geo_metadata_df.columns)-4, len(geo_metadata_df.columns)):
    new_column_list.append(geo_metadata_df.columns[j])
geo_metadata_df.columns = new_column_list

new_column_list = []
new_column_list.append(geo_df.columns[len(geo_df.columns)-5])
for i in range(0,len(geo_df.columns)-5):
    new_column_list.append(geo_df.columns[i])
for j in range(len(geo_df.columns)-4, len(geo_df.columns)):
    new_column_list.append(geo_df.columns[j])



In [ ]:
varlist = []
string1 = 'Baltimore'
#string2 = 'Male'
#string3 = ' '

#string1 = 'TRAVEL TIME
#string2 = '10 to 14 Minutes
#string3 = 'bus'
#string1 = 'MALE'
string2 = ' '
string3 = ' '
#string2 = 'all'
#string3 = ''

geo_df['NAME'][geo_df['NAME'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x))]

In [ ]:
print('Females under age 5 in Baltimore County: {0:,.0f} with margin of error {1:,.0f}'.format(data_df['B01001_027'].loc['05000US24510'], error_df['B01001_027'].loc['05000US24510']))
#data_df['B01001_027'].loc['05000US24510']
#error_df['B01001_027'].loc['05000US24510']